In [2]:
import pandas as pd 
from sklearn import tree
import numpy as np
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/GraphViz'

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
model.fit(X, y)
predictions = model.predict(X_test)

In [3]:
def clean_df(df):
    df["SibSp"] = df["SibSp"] + df["Parch"]
    df = df.replace({"Sex": {'male': 1, 'female': 0}})

    del df["Name"]
    del df["Parch"]
    del df["Ticket"]
    del df["Embarked"]
    del df["PassengerId"]
    del df["Cabin"]
    # I have doubts about this one
    # del df["Fare"]

    df = df.fillna(df.mean().astype(int))

    return df


In [4]:
train = pd.read_csv("titanic/input/train.csv")
train = clean_df(train)
train_y = train.pop("Survived")

test = pd.read_csv("titanic/input/test.csv")
test = clean_df(test)

answer = pd.read_csv("titanic/input/gender_submission.csv")
del answer['PassengerId']

In [5]:
def create_train(train, y, criterion, max_depth, test):
    clf = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
    clf.fit(train, y)
    if max_depth:
        file_name = "titanic/output/" + criterion + str(max_depth)
    else:
        file_name = "titanic/output/" + criterion
    dot_data = tree.export_graphviz(clf, out_file=None, 
                      feature_names=train.columns,
                      class_names=["Didn't", "Survived"],
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(dot_data)
    graph.render(file_name)
    result = clf.predict(test)
    out_df = pd.DataFrame(data=result, columns=["Survived"])
    out_df.to_csv(file_name + ".csv")
    return out_df

In [6]:
d1 = create_train(train, train_y, "gini", None, test)
d2 = create_train(train, train_y, "gini", 4, test)
d3 = create_train(train, train_y, "entropy", None, test)
d4 = create_train(train, train_y, "entropy", 4, test)

In [7]:
def accuracy(df, answer):
    acc = (df == answer).value_counts()[True]
    acc = int(acc) / len(d1)
    return acc

In [8]:
print("Accuracy for gini - unlimited depth is %s" % accuracy(d1, answer))
print("Accuracy for gini - 4 depth is %s" % accuracy(d2, answer))
print("Accuracy for entropy - unlimited depth is %s" % accuracy(d3, answer))
print("Accuracy for entropy - 4 depth is %s" % accuracy(d4, answer))

Accuracy for gini - unlimited depth is 0.8038277511961722
Accuracy for gini - 4 depth is 0.9593301435406698
Accuracy for entropy - unlimited depth is 0.7942583732057417
Accuracy for entropy - 4 depth is 0.9401913875598086


In [9]:
test = pd.read_csv("titanic/input/test.csv")
del test["Name"]
del test["Parch"]
del test["Ticket"]
del test["Embarked"]
del test["Cabin"]
# I have doubts about this onea
del test["Fare"]
del test["Pclass"]
del test["Sex"]
del test["Age"]
del test["SibSp"]
d2, test

a = test.join(d1)
a.to_csv("kaggle-3.csv", index=False)